In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
*   если источник traffic_source равен yandex или google, то в source_type ставится organic
*   для источников paid и email из России - ставим ad
*   для источников paid и email не из России - ставим other
*   все остальные варианты берем из traffic_source без изменений





In [ ]:
df_visit = pd.read_csv('/content/drive/MyDrive/visit_log.csv', sep=';')
df_visit.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [ ]:
def source_type(row):
    if row['traffic_source'] == 'yandex' or row['traffic_source'] == 'google':
        return 'organic'
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email') and row['region'] == 'Russia':
        return 'ad'
    elif (row['traffic_source'] == 'paid' or row['traffic_source'] == 'email') and row['region'] != 'Russia':
        return 'other'
    else:
        return row['traffic_source']

In [ ]:
df_visit['source_type'] = df_visit.apply(source_type, axis=1)
df_visit.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


# Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
* Прочитайте содержимое файла с датафрейм
* Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
df_url = pd.read_csv('/content/drive/MyDrive/URLs.txt')
df_url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [ ]:
regexp = r'(/\d{8}-)'

In [ ]:
df_url[df_url.url.str.contains(regexp)].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
df_ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
life = df_ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'userId': 'count'})
life.head()

timestamp             userId
               min         max  count
userId                               
1       1260759108  1260759205     20
2        835355395   835356246     76
3       1298861589  1298932787     51
4        949778714   949982274    204
5       1163373044  1163375145    100

In [ ]:
life = life[life['userId']['count'] > 100]

In [ ]:
life.loc[:, 'livetime'] = life['timestamp']['max'] - life['timestamp']['min']

In [ ]:
life.reset_index().head()

userId   timestamp             userId   livetime
                 min         max  count           
0      4   949778714   949982274    204     203560
1      8  1154389340  1154474527    116      85187
2     15   997937239  1469330735   1700  471393496
3     17  1127468587  1127476640    363       8053
4     19   855190091   855195373    423       5282

# Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).

Необходимо сформировать две таблицы:
* таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
* аналогичную таблицу по типам выручки с указанием адреса клиента

In [ ]:
df_rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
df_rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [ ]:
df_auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
df_auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [ ]:
df_air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
df_air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [ ]:
df_client = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
df_client

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [ ]:
df_client_without = df_client.drop('address',1)
df_client_without

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,client_id
0,111
1,112
2,113
3,114
4,115
5,116
6,117
7,118


In [ ]:
df_result = df_client_without.merge(df_rzd, how='left', on='client_id')
df_result = df_result.merge(df_auto, how='left', on='client_id')
df_result = df_result.merge(df_air, how='left', on='client_id')
df_result

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [ ]:
df_resul_addr = df_result.merge(df_client, how='left', on='client_id')
df_resul_addr

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,57483.0,NaN,Левобережная 1а
3,114,5774.0,83.0,NaN,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,NaN,4834.0,4.0,Строителей 18
6,117,NaN,98.0,13.0,Панфиловская 33
7,118,NaN,NaN,173.0,Мастеркова 4


In [ ]:
df_result.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [ ]:
df_resul_addr.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
